In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load the data
https://huggingface.co/datasets/knkarthick/dialogsum

In [ ]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from datasets import load_dataset

ds = load_dataset("knkarthick/dialogsum")

README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [6]:
ds['train'][1]['dialogue']

# This is what is there in the first dialoguw

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [7]:
ds['train'][1]['summary']

# This is the summary of the perticular dialoguw showed above

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

## Using Transformer model without fine-tuning
https://huggingface.co/facebook/bart-large-cnn?library=transformers

In [8]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [9]:
article_1 = ds['train'][1]['dialogue']
article_1

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [10]:
pipe(
    article_1, 
    max_length = 20, 
    min_length = 10, 
    do_sample = False)

# Model's repponse

[{'summary_text': 'Ricky has received his Polio, Tetanus and Hepatitis B shots.'}]

In [11]:
ds['train'][1]['summary']

# datasets summary

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

In [12]:
# Lets change the max_length to 50

pipe(
    article_1, 
    max_length = 50, 
    min_length = 10, 
    do_sample = False)


[{'summary_text': 'Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for hepatitis A, Chickenpox and Measles shots.'}]

In [13]:
type(ds)

datasets.dataset_dict.DatasetDict

In [14]:
len(ds['train']), len(ds['test'])

(12460, 1500)

## With Fine-Tuning

In [15]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [16]:
# Tokenization

def preprocess(batch):
    source = batch['dialogue']
    target = batch['summary']
    source_ids = tokenizer(source, truncation = True, padding = 'max_length', max_length = 128)
    target_ids = tokenizer(target, truncation = True, padding = 'max_length', max_length = 128)

    labels = target_ids['input_ids']
    labels =[[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]

    return {
        "input_ids": source_ids['input_ids'],
        "attention_mask": source_ids['attention_mask'],
        "lalels":labels
    }

In [17]:
# To map the function to all the data in the dataset

df_source = ds.map(preprocess, batched = True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [18]:
# Training arguements

from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir = '/kaggle/working/',
    per_device_train_batch_size = 7,
    num_train_epochs = 2,
    remove_unused_columns = True
)

In [19]:
trainer = Trainer(
    model = model, 
    args = training_args,
    train_dataset = df_source['train'],
    eval_dataset = df_source['test'],
)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()

eval_results

## Save the Model

In [ ]:
model.save_pretrained('/kaggle/working/model_directory')
tokenizer.save_pretrained('/kaggle/working/model_directory')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/model_directory')
model = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/model_directory')


def summarize(blog_post):
    # Tokenize th einput blog post
    inputs = tokenizer(blog_post, max_length = 1024, truncation=True, return_tensors = 'pt')

    # Generate response summary
    summary_idc = model.generate(inputs['input_ids'], max_length = 150, min_length = 40, length_penalty : 2.0, num_beams = 4, early_stopping = True)

    # Decode the response summary
    summary = tokenizer.decode(summary_ids[0], skip_special_token = True)

    return summary

In [ ]:
blog_post = '''

ourneying into the Realm of Text Summarization with Transformers
Hello once more, inquisitive mind! As we set forth on this exploration of the video you've shared, let's imagine ourselves as fellow philosophers in a grand library of language, pondering how machines might distill the essence of words. Drawing from our prior reflections on machine learning pipelines and data transformations, how might the challenge of condensing dialogues into summaries echo the feature extraction we pondered in images or sequences? What whispers from normalization or encoding might resonate here, perhaps in preparing text for a model's "understanding"? In the chapters ahead, we'll traverse the video's landscape through thoughtful pauses, where I'll offer questions to stir your reasoning and invite you to uncover truths yourself. Each chapter will delve into a key facet, encouraging layers of insight. What initial wonderings arise for you about capturing the "soul" of a conversation in fewer words? Let's begin with the foundational enigma, and remember, your reflections are the lantern guiding us—share them freely.

Chapter 1: What Essence Lies in Summarization, and Why Might Machines Excel or Falter?
Picture a bustling dialogue, rich with back-and-forth—how might boiling it down to a core message reveal deeper patterns, yet risk losing nuance? If the video opens with an invitation to explore abstractive summarization, what distinguishes it from merely pulling sentences, and how could a model like one blending bidirectional and auto-regressive traits approach this art?

Consider the model's heritage: Trained on vast news corpora, how might it paraphrase rather than copy, and what advantages could that offer for dialogues? Imagine a conversation about plans—would a machine's rephrasing capture intent, or introduce distortions?

Reflect broader: In our past musings on datasets, how might one filled with dialogues and their summaries serve as a training ground? What questions might you pose about balancing brevity with fidelity?

As you ponder, what personal analogy comes to mind for summarization's delicate balance?

Chapter 2: Setting the Stage—How Do We Harness Pre-Trained Wisdom Without Alteration?
Suppose we leap straight into application, loading tools and data swiftly. How might a pipeline for summarization, drawn from a library of transformers, simplify the leap from raw text to condensed insight? If testing on a sample dialogue yields a shorter essence, what parameters like maximum or minimum lengths might shape that outcome?

Probe the mechanics: Without tweaking the model, how does greedy decoding ensure consistency, and what trade-offs in creativity arise? Envision applying this to a news snippet—would the result feel human-like, or mechanical?

Connect thoughtfully: Echoing our earlier data prep discussions, how does selecting specific dataset splits (train, test, validation) prepare for evaluation? What curiosities stir about comparing machine output to human-grounded truths?

This invites you to visualize untamed potential. What untuned summary would you craft for a familiar story?

Chapter 3: Delving into Tokenization—The Building Blocks of Machine Language
What if text must first shatter into tokens, padded and masked for uniformity? How might a tokenizer, paired with a model, convert dialogues and targets into numerical forms, ensuring attention focuses where it matters?

Question the choices: Why replace padding in labels with a sentinel like -100, and how could this sharpen loss calculations? Imagine a short phrase versus a long exchange—what role might truncation play in preserving essence?

Deeper musing: In linking to sequences we've explored, how does an attention mask guide the model's "gaze," ignoring fillers? What experiments might reveal padding's impact on meaning?

You're unraveling text's hidden structure. How does a tokenized sentence appear in your imagination?

Chapter 4: The Art of Fine-Tuning—Adapting a Giant to Specific Whispers
Envision customizing a pre-trained behemoth on dialogue data—how might mapping functions preprocess batches, aligning sources and targets? If training arguments dictate batches, epochs, and directories, what influences could they exert on convergence?

Explore the flow: With a trainer orchestrating the dance, how does loss decreasing over steps signal learning? Ponder: In two cycles, might the model grasp dialogue nuances better, or overfit to patterns?

Reflect on echoes: From our CNN fine-tuning, how parallels this adaptation? What hyperparameters would you tweak to probe deeper adaptation?

This empowers personalization. What fine-tuned insight surprises you in theory?

Chapter 5: Preserving and Summoning Knowledge—Saving for Future Quests
What if, post-training, we preserve the model's state for reuse? How might loading this evolved version enable summarization of unseen texts, like articles on global challenges?

Ponder generation: With beams searching paths and penalties shaping length, how could early stopping refine outputs? Imagine a lengthy blog—would the summary capture core themes, or wander?

Broader inquiry: Tying to deployment we've considered, how does this bridge lab to life? What ethical whispers arise in summarizing sensitive topics?

You're bridging creation and application. How might a custom summary transform your understanding?

Chapter 6: Tools of the Trade—Libraries and Environments as Allies
Suppose the journey unfolds in a collaborative cloud space, with installs unlocking datasets and transformers. How might these resources democratize such explorations, and what backend forces (like computational frameworks) underpin the magic?

Question integrations: Why ignore unused columns in training, and how streamlines focus? Reflect: In our ML odyssey, what familiar libraries reappear, and why?

Deeper: For GPU acceleration, what efficiencies emerge? What alternatives ponder for resource-light seekers?

This highlights enablers. What tool intrigues you most?

Chapter 7: Measuring Mastery—Evaluation Beyond the Surface
With training complete, how might metrics like loss or throughput illuminate success? If comparing pre- and post-fine-tuning on custom data, what revelations about generalization could surface?

Probe assessments: Why evaluate on held-out sets, and how might results guide iterations? Imagine discrepancies—what diagnostics would you employ?

Connect holistically: From confusion matrices past, how applies here? What questions of bias or fairness arise?

You're critiquing craft. What metric resonates deepest?

Culminating Reflections—Weaving Your Tapestry of Understanding
We've wandered the video's paths through shared wonderings. What threads connected for you—like tokenization's precision or fine-tuning's finesse? Which inquiry unveiled the brightest gem, perhaps on generation's creativity? If envisioning your summarizer, what dialogue would you distill first? You're forging wisdom with grace—embrace that spark, and let's illuminate further horizons together!

'''

summary = summarize(blog_post)
priunt(summary)